In [13]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from mlxtend.frequent_patterns import apriori, association_rules

In [14]:
# Function to reduce the memory usage of a DataFrame.
def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# Generator function to load data in chunks.
def data_generator(df, chunksize=10000):
    for i in range(0, df.shape[0], chunksize):
        yield df.iloc[i:i+chunksize]

In [15]:
df = reduce_memory(pd.read_csv("data/Dataset.csv"))

In [16]:
df

,UserID,Game,purchase/play,Heure_jouee,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,...,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Score
0,151603712,The Elder Scrolls V Skyrim,play,273.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,107.633331,9.933333,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0
1,59945701,The Elder Scrolls V Skyrim,play,58.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,107.633331,9.933333,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",2.5
2,92107940,The Elder Scrolls V Skyrim,play,110.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,107.633331,9.933333,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0
3,250006052,The Elder Scrolls V Skyrim,play,465.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,107.633331,9.933333,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0
4,11373749,The Elder Scrolls V Skyrim,play,220.0,72850,"Nov 10, 2011",5000000 - 10000000,4383,17,19.99,...,107.633331,9.933333,50.533333,14.633333,Bethesda Game Studios,Bethesda Softworks,"Single-player,Steam Achievements,Steam Trading...",RPG,"Open World,RPG,Fantasy,Adventure,Dragons,Modda...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56784,116564064,The Bug Butcher,play,0.3,350740,"Jan 19, 2016",100000 - 200000,1,0,7.99,...,1.833333,0.000000,2.516667,0.000000,Awfully Nice Studios,Awfully Nice Studios,"Single-player,Multi-player,Co-op,Shared/Split ...","Action,Indie","Indie,Action,Shoot 'Em Up,Arcade,Local Co-Op,C...",0.5
56785,72417734,Romance of the Three Kingdoms Maker,play,0.3,397720,"Dec 9, 2015",500000 - 1000000,7,0,0.00,...,2.616667,0.000000,0.016667,0.000000,"KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.","Single-player,Includes level editor",Simulation,"Simulation,Strategy",0.5
56786,135400225,Life is Hard,play,3.1,414080,"Aug 14, 2021",50000 - 100000,2,0,9.99,...,5.033333,0.000000,5.366667,0.000000,"Pirozhok Studio,Vitaliy Ruzankin",Pirozhok Studio,"Single-player,Steam Achievements,Steam Trading...","Adventure,Indie,Simulation,Strategy","Strategy,Indie,Simulation,Adventure,Early Acce...",3.0
56787,135400225,Executive Assault,play,2.7,331500,"Jul 17, 2015",20000 - 50000,1,0,12.99,...,1.633333,0.000000,1.633333,0.000000,Hesketh Studios Ltd,Hesketh Studios Ltd,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Action,Strategy","Strategy,RTS,Action,FPS,Base-Building,Sci-fi,R...",5.0


In [17]:
df[df['Average playtime forever'] < 0.1]

,UserID,Game,purchase/play,Heure_jouee,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,...,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Score
11320,59945701,Sid Meier's Civilization V,play,22.0,50100,"Sep 21, 2010",0 - 20000,0,0,29.99,...,0.000000,0.0,0.000000,0.0,"Firaxis Games,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)","Single-player,Multi-player,Steam Achievements,...",Strategy,"Turn-Based Strategy,Strategy,Turn-Based,Multip...",5.0
11322,53875128,Sid Meier's Civilization V,play,5.9,50100,"Sep 21, 2010",0 - 20000,0,0,29.99,...,0.000000,0.0,0.000000,0.0,"Firaxis Games,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)","Single-player,Multi-player,Steam Achievements,...",Strategy,"Turn-Based Strategy,Strategy,Turn-Based,Multip...",5.0
11324,56038151,Sid Meier's Civilization V,play,72.0,50100,"Sep 21, 2010",0 - 20000,0,0,29.99,...,0.000000,0.0,0.000000,0.0,"Firaxis Games,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)","Single-player,Multi-player,Steam Achievements,...",Strategy,"Turn-Based Strategy,Strategy,Turn-Based,Multip...",5.0
11326,57103808,Sid Meier's Civilization V,play,35.0,50100,"Sep 21, 2010",0 - 20000,0,0,29.99,...,0.000000,0.0,0.000000,0.0,"Firaxis Games,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)","Single-player,Multi-player,Steam Achievements,...",Strategy,"Turn-Based Strategy,Strategy,Turn-Based,Multip...",5.0
11328,63276972,Sid Meier's Civilization V,play,79.0,50100,"Sep 21, 2010",0 - 20000,0,0,29.99,...,0.000000,0.0,0.000000,0.0,"Firaxis Games,Aspyr (Mac),Aspyr (Linux)","2K,Aspyr (Mac),Aspyr (Linux)","Single-player,Multi-player,Steam Achievements,...",Strategy,"Turn-Based Strategy,Strategy,Turn-Based,Multip...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56775,60859695,Nancy Drew The Creature of Kapu Cave,play,1.6,31900,"Jul 17, 2009",20000 - 50000,0,0,6.99,...,0.000000,0.0,0.000000,0.0,HeR Interactive,HeR Interactive,Single-player,Adventure,"Adventure,Point & Click,Puzzle,Mouse only,Dete...",5.0
56777,75970863,Diner Dash Hometown Hero,play,5.1,37240,"Aug 12, 2009",0 - 20000,9,0,9.99,...,0.083333,0.0,0.083333,0.0,PlayFirst,PlayFirst,Single-player,Simulation,"Time Management,Casual,Cooking",5.0
56778,75970863,DinerTown Tycoon,play,1.0,37300,"Sep 30, 2009",0 - 20000,0,0,9.99,...,0.000000,0.0,0.000000,0.0,PlayFirst,PlayFirst,Single-player,Simulation,"Casual,Management",5.0
56779,42657809,Kohan II Kings of War,play,2.3,97130,"Aug 24, 2011",20000 - 50000,7,13,9.99,...,0.083333,0.0,0.150000,0.0,TimeGate Studios,TimeGate Studios,"Single-player,Multi-player,Co-op",Strategy,"Strategy,RTS,Fantasy,Classic",5.0
